In [ ]:
USE dbSigaSalesianoSaoJose_Arquivo05

   SELECT file_id, name as [logical_file_name],physical_name from sys.database_files

   DBCC SHRINKFILE (dbSigaSalesianoSaoJose_Arquivo05_log , 0, TRUNCATEONLY) 

In [ ]:
         DECLARE @BatchSize INT = 100000       
        
        WHILE 1 = 1
        BEGIN
            DELETE TOP (@BatchSize)
            FROM  dbSigaNeves_Arquivo05.dbo.TbFaseNotaAluno_Auditoria  
            WHERE year(DataHora) < ( year( getdate() ) - 2 )
        
            IF @@ROWCOUNT < @BatchSize BREAK
        END

In [ ]:
EXEC master.dbo.sp_foreachdb N'
        USE [?]

        DELETE  FROM ?.dbo.TbFaseNotaAluno_Auditoria  WHERE year(DataHora) < ( year( getdate() ) - 2 )

        DELETE  FROM ?.dbo.TbSituacaoAlunoDisciplina_Auditoria  WHERE year(DataHora) < ( year( getdate() ) - 2 )

        DELETE  FROM ?.dbo.TbMetaSituacaoAlunoDisciplina_Auditoria  WHERE year(DataHora) < ( year( getdate() ) - 2 )

        DELETE  FROM ?.dbo.TbAuditoria  WHERE Tabela IN (''Sistema_Login'',''Sistema_LogOut'',''TbDiarioFrequencia'',''TbDiarioAula'',''TbDiario'',''TbFaseNota'',''TbDiarioAluno'') AND year(DataHora) < ( year( getdate() ) - 2 )

        DELETE  FROM ?.dbo.TbAlunoTurmaProcedimentoMatriculaHistorico  WHERE year(DataAtualizacao) < ( year( getdate() ) - 2 )

        DELETE  FROM ?.dbo.TbLogWeb  WHERE year(DataHora) < ( year( getdate() ) - 2 )

        DELETE  FROM ?.dbo.TbLogMobile  WHERE year(DataHoraLogin) < ( year( getdate() ) - 2 )

        DELETE  FROM ?.dbo.TbMensagemCaixaSaida  WHERE year(DataHoraInsercao) < ( year( getdate() ) - 2 )

    SELECT file_id, name as [logical_file_name],physical_name from sys.database_files

           IF CHARINDEX( ''dbSiga'', ''?'') > 0
       DBCC SHRINKFILE (dbSiga_Log , 0, TRUNCATEONLY) 
   
   IF CHARINDEX( ''dbSigaAuxilNatal'', ''?'') > 0
       DBCC SHRINKFILE (teste2_Log , 0, TRUNCATEONLY) 

   ELSE IF ( CHARINDEX( ''dbActiveBib'', ''?'') > 0 ) OR ( CHARINDEX( ''dbBib'', ''?'') > 0 )
       DBCC SHRINKFILE (dbSigaBiblioteca_Log , 0, TRUNCATEONLY) 
   
   ELSE IF ( CHARINDEX( ''dbCantina'', ''?'') > 0 ) OR ( CHARINDEX( ''dbActive'', ''?'') > 0 )
       DBCC SHRINKFILE (dbCantinaActive_log , 0, TRUNCATEONLY) 
   
   ELSE IF ( CHARINDEX( ''dbCrmActivesoft'', ''?'') > 0 ) 
       DBCC SHRINKFILE (dbActiveCRM2_Log , 0, TRUNCATEONLY) 
   
   ELSE IF CHARINDEX( ''CRM_'', ''?'') > 0
       DBCC SHRINKFILE (dbSiga_Log , 0, TRUNCATEONLY) 
'
,@print_command_only = 0 -- NAO Exec direto
,@print_dbname=0
,@state_desc = N'ONLINE'
,@user_only = 1
,@suppress_quotename=1
,@name_pattern='CRM_dbCrm233704v3';
-- ,@database_list = 'dbSigaLis'

In [ ]:
-- ROLLBACK
-- 
COMMIT

In [ ]:
-- SHRINKDATABASE sp_foreachdb  (Joir Ramalho v1.00 11Jan21)

EXEC master.dbo.sp_foreachdb N'
    CALL BACKUPbanco.bat full ?
'
,@print_command_only = 1 -- Obrigatório Gerar script
,@print_dbname=0
,@state_desc = N'ONLINE'
,@user_only = 1
,@suppress_quotename=1
,@name_pattern='dbSigaNEVES_ARQ';
-- ,@database_list = 'dbSigaAprendiz'  --  -- dbSigaEscolaNovaCruz, dbSigaDom, dbSigaNSAparecida, dbSigaDesafio, dbSigaDomMarcolino, dbSigaGandhi, dbSigaCristoReiMaceio, dbSigaSantAna,dbSigaCarochinha,dbSigaCoesi

## Limpeza Cantina

In [ ]:
-- Script SELECT
EXEC master.dbo.sp_foreachdb N'
    USE [?]

    SELECT Tabela, year( DataHora ), count(*)	FROM ?.dbo.TbAuditoria  WHERE   Tabela IN (''TbCaixaLancamento'',''TbCaixaMovimentacao'',''TbConta'',''TbLancamentoConsumo'',''TbLancamentoConsumoItem'',''TbLancamentoCredito'',''TbProdutoEstoque'') group by Tabela, year( DataHora )        order by year( DataHora ) DESC
'
,@print_command_only = 0
,@print_dbname=0
,@state_desc = N'ONLINE'
,@user_only = 1
,@suppress_quotename=1
,@name_pattern='dbActiveCantina';
-- ,@database_list = 'dbCantina';

In [ ]:
-- Script INSERT INTO
EXEC master.dbo.sp_foreachdb N'
    ALTER DATABASE [?] SET RECOVERY SIMPLE WITH NO_WAIT

    BEGIN TRAN
        SELECT  * INTO VPS1_20210123_Dados2018.dbo.?_TbAuditoria FROM ?.dbo.TbAuditoria WHERE Tabela IN (''TbCaixaLancamento'',''TbCaixaMovimentacao'',''TbConta'',''TbLancamentoConsumo'',''TbLancamentoConsumoItem'',''TbLancamentoCredito'',''TbProdutoEstoque'') AND year(DataHora) < ( year( getdate() ) - 1 )

       DELETE  FROM ?.dbo.TbAuditoria  WHERE Tabela IN (''TbCaixaLancamento'',''TbCaixaMovimentacao'',''TbConta'',''TbLancamentoConsumo'',''TbLancamentoConsumoItem'',''TbLancamentoCredito'',''TbProdutoEstoque'') AND year(DataHora) < ( year( getdate() ) - 1 )
    COMMIT

    ALTER DATABASE [?] SET RECOVERY FULL WITH NO_WAIT
'
,@print_command_only=0
,@print_dbname=0
,@state_desc = N'ONLINE'
,@user_only = 1
,@suppress_quotename=1
,@name_pattern='dbActiveCantina';
--,@database_list = 'dbActiveCantinaCIEC';